In [ ]:
import time
x = [1,2,3]
z=[1]
y=[]
for i in range(10000):
    y.append(x)
    x+=z
time1 = time.time()
target_spans = set([str(span) for span in y])
print(time.time()-time1)
time1 = time.time()
result_set = set(map(str, y))
print(time.time()-time1)

In [ ]:
import sys
sys.path.append('../')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import warnings
warnings.filterwarnings('ignore')
from data.pipe import BartNERPipe
from model.bart import BartSeq2SeqModel
import fitlog
import random
import numpy as np

import torch
from fastNLP import Trainer
from model.metrics import Seq2SeqSpanMetric
from model.losses import Seq2SeqLoss
from torch import optim
from fastNLP import BucketSampler, GradientClipCallback, cache_results

from model.callbacks import WarmupCallback
from fastNLP.core.sampler import SortedSampler
from model.generater import SequenceGeneratorModel
from fastNLP.core.sampler import  ConstTokenNumSampler
from model.callbacks import FitlogCallback

fitlog.debug()
fitlog.set_log_dir('logs')

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset_name', default='en_ace04', type=str)

def set_seed(seed=1996):
    print("[SET SEED]: ",seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

args= parser.parse_args([])
dataset_name = args.dataset_name
args.length_penalty = 1
args.save_model = 1

# word: 生成word的start; bpe: 生成所有的bpe; span: 每一段按照start end生成; span_bpe: 每一段都是start的所有bpe，end的所有bpe
args.target_type = 'word'
args.bart_name = '/disk1/wxl/Desktop/DeepKE/example/ner/huggingface/bart-large'
args.schedule = 'linear'
args.decoder_type = 'avg_feature'
args.n_epochs = 30
args.num_beams = 1
args.batch_size = 16
args.use_encoder_mlp = 1
args.lr = 1e-5
args.warmup_ratio = 0.01
eval_start_epoch = 1

# the following hyper-parameters are for target_type=word
if dataset_name == 'conll2003':  # three runs get 93.18/93.18/93.36 F1
    max_len, max_len_a = 10, 0.6
elif dataset_name == 'en-ontonotes':  # three runs get 90.46/90.4/90/52 F1
    max_len, max_len_a = 10, 0.8
elif dataset_name == 'CADEC':
    max_len, max_len_a = 10, 1.6
    args.num_beams = 4
    args.lr = 2e-5
    args.n_epochs = 30
    eval_start_epoch=10
elif dataset_name == 'Share_2013':
    max_len, max_len_a = 10, 0.6
    args.use_encoder_mlp = 0
    args.num_beams = 4
    args.lr = 2e-5
    eval_start_epoch = 5
elif dataset_name == 'Share_2014':
    max_len, max_len_a = 10, 0.6
    args.num_beams = 4
    eval_start_epoch = 5
    args.n_epochs = 30
elif dataset_name == 'genia':  # three runs: 79.29/79.13/78.75
    max_len, max_len_a = 10, 0.5
    args.target_type = 'span'
    args.lr = 2e-5
    args.warmup_ratio = 0.01
elif dataset_name == 'en_ace04':  # four runs: 86.84/86.33/87/87.17
    max_len, max_len_a = 50, 1.1
    args.n_epochs = 55
    args.batch_size = 48
    args.lr = 4e-5
    seed = 4373
elif dataset_name == 'en_ace05':  # three runs: 85.39/84.54/84.75
    max_len, max_len_a = 50, 0.7
    args.lr = 3e-5
    args.batch_size = 12
    args.num_beams = 4
    args.warmup_ratio = 0.1

set_seed(seed)
# with open("/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/loss_log/D.json","r") as f:
#     b=f.readlines()
# with open("/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/loss_log/E.json","r") as f:
#     c=f.readlines()
# for x,y in zip(b,c):
#     assert x==y,print(x,y)
# exit()

save_model = args.save_model
del args.save_model
lr = args.lr
n_epochs = args.n_epochs
batch_size = args.batch_size
num_beams = args.num_beams

length_penalty = args.length_penalty
if isinstance(args.decoder_type, str) and args.decoder_type.lower() == 'none':
    args.decoder_type = None
decoder_type = args.decoder_type
target_type = args.target_type
bart_name = args.bart_name
schedule = args.schedule
use_encoder_mlp = args.use_encoder_mlp

fitlog.add_hyper(args)

#######hyper
#######hyper

demo = False
if demo:
    cache_fn = f"caches/data_{bart_name}_{dataset_name}_{target_type}_demo.pt"
else:
    cache_fn = f"caches/data_{bart_name}_{dataset_name}_{target_type}.pt"

@cache_results(cache_fn, _refresh=False)
def get_data():
    pipe = BartNERPipe(tokenizer=bart_name, dataset_name=dataset_name, target_type=target_type)
    if dataset_name == 'conll2003':
        paths = {'test': "./data/conll2003/test.txt",
                 'train': "./data/conll2003/train.txt",
                 'dev': "./data/conll2003/dev.txt"}
        data_bundle = pipe.process_from_file(paths, demo=demo)
    elif dataset_name == 'en-ontonotes':
        paths = './data/en-ontonotes/english'
        data_bundle = pipe.process_from_file(paths)
    else:
        data_bundle = pipe.process_from_file(f'./data/{dataset_name}', demo=demo)
    return pipe, data_bundle, pipe.tokenizer, pipe.mapping2id


In [ ]:
pipe, data_bundle, tokenizer, mapping2id = get_data()
ds = data_bundle.get_dataset("train")
len(ds)

In [ ]:
x=0
is_key = set()
for id,i in enumerate(ds):
    # x = []
    # for j in i["target_span"]:
        
    #     if j not in x:
    #         x.append(j)
    #     else:
    #         print(i["tgt_tokens"])
    #         print(i["raw_words"])
    #         print(id)
    if len(i["target_span"])==0:
        is_key.add(str(i["raw_words"]))
print(len(is_key))

In [ ]:
from copy import deepcopy 
from fastNLP import DataSet
def current_tree_init(ds):
    num = 0
    num2 = 0
    num_x = 0
    num_zero_ent = 0
    ds_ = DataSet()

    current_tree = {}

    is_ordered_key = set()

    split_list = ["entities","entity_tags","entity_spans","tgt_tokens","target_span","tgt_seq_len"]
    
    for id, ds_i in enumerate(ds):
        before = {}

        for i in split_list:
            before[i]=ds_i[i]

        num2 += len(before["entity_tags"])
        key = ' '.join(ds_i["raw_words"])
        if key in current_tree.keys():
            continue # 重复数据
        value = {
            "ds_indexes": [],
            "position": 1,
            "stop_position": ds_i["tgt_seq_len"]
        }
        '''
            [position]: 未排序的第一个位置
            [stop_position]: 排序结束的位置，初始化为key对应的原始tgt_seq_len
        '''
        
        num+=len(before["entity_tags"])
        if len(before["entity_tags"]) <=1:
            num_x += 1 

        for i in range(len(before["entity_tags"])):
            split_i = {
                "entities":before["entities"][i:i+1],
                "entity_tags":before["entity_tags"][i:i+1],
                "entity_spans":before["entity_spans"][i:i+1],
                "target_span":before["target_span"][i:i+1]
            }
            split_i["tgt_tokens"] = [0] + split_i["target_span"][0] + [1]
            split_i["tgt_seq_len"] = len(split_i["tgt_tokens"])

            x = deepcopy(ds_i)
            for j in split_list:
                x[j] = split_i[j]

            ds_.append(x)

            value["ds_indexes"].append(len(ds_)-1)

        if len(value["ds_indexes"]) <= 1:
            value["position"] = value["stop_position"] # 0个或1个实体，不用排序
            is_ordered_key.add(key)
            if len(value["ds_indexes"]) == 0:
                num_zero_ent += 1
                x = deepcopy(ds_i)
                ds_.append(x) # 0个实体，直接将原数据加回去
                value["ds_indexes"].append(len(ds_)-1)
        current_tree[key] = value # 处理完之后加入字典树
        

    print("非去重实体个数",num2," 去重实体个数： ",num," 新训练数据个数：",len(ds_)," 不重复数据个数: ", len(current_tree), " 0实体数据个数：",num_zero_ent)
    print(num_x)
    assert len(ds_) == num+num_zero_ent
    return ds_, current_tree, is_ordered_key


In [ ]:
ds_,current_tree, is_ordered_key = current_tree_init(ds)
print(len(ds_))
ds_.set_ignore_type('target_span', 'entities')
ds_.set_pad_val('tgt_tokens', 1)  # 设置为eos所在的id
ds_.set_pad_val('src_tokens', pipe.tokenizer.pad_token_id)

ds_.apply_field(lambda x: len(x), field_name='src_tokens', new_field_name='src_seq_len')
ds_.apply_field(lambda x: len(x), field_name='tgt_tokens', new_field_name='tgt_seq_len')
ds_.set_input('tgt_tokens', 'src_tokens', 'src_seq_len', 'tgt_seq_len', 'first')
ds_.set_target('tgt_tokens', 'tgt_seq_len', 'target_span', 'entities')


In [ ]:
print(f'max_len_a:{max_len_a}, max_len:{max_len}')

print(data_bundle)
print("The number of tokens in tokenizer ", len(tokenizer.decoder))

bos_token_id = 0
eos_token_id = 1
label_ids = list(mapping2id.values())
model = BartSeq2SeqModel.build_model(bart_name, tokenizer, label_ids=label_ids, decoder_type=decoder_type,
                                     use_encoder_mlp=use_encoder_mlp)

vocab_size = len(tokenizer)
print(vocab_size, model.decoder.decoder.embed_tokens.weight.data.size(0))
model = SequenceGeneratorModel(model, bos_token_id=bos_token_id,
                               eos_token_id=eos_token_id,
                               max_length=max_len, max_len_a=max_len_a,num_beams=num_beams, do_sample=False,
                               repetition_penalty=1, length_penalty=length_penalty, pad_token_id=eos_token_id,
                               restricter=None)

In [ ]:
model2=torch.load("/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/save_models/en_ace04_3257/best_SequenceGeneratorModel_f_2024-11-18-22-16-54-082796").to('cuda')

In [ ]:
def check_final_data(current_tree, ds, ds2):
    num_ent = 0
    for ds_i in ds:
    # 映射流程：ds[i] -> key -> current_tree_final[key] -> ds2[de_indexes[0]]
        key = ' '.join(ds_i["raw_words"])
        ds2_i = ds2[current_tree[key]["ds_indexes"][0]]
        assert len(current_tree[key]["ds_indexes"])==1
    # assert: tgt_seq_len == len(tgt_tokens), tgt_tokens == [0] + Σtatget_span[i] + [1]
        assert ds2_i["tgt_seq_len"] == len(ds2_i["tgt_tokens"])
        tmp = []
        for i in ds2_i["target_span"]:
            tmp += i
        assert ds2_i["tgt_tokens"] == [0] + tmp + [1]
    # ds_set()/ds2_set(): str(entities[i]+entity_tags[i]+entity_spans[i]+target_span[i]) 
        ds_set = set()
        ds2_set = set()
        num_ent += len(ds2_i["entities"]) 
        assert len(ds2_i["entities"]) == len(ds2_i["entity_tags"]) == len(ds2_i["entity_spans"]) == len(ds2_i["target_span"])
        for i in range(len(ds2_i["entities"])):
            v = str(ds2_i["entities"][i]) + str(ds2_i["entity_tags"][i]) + str(ds2_i["entity_spans"][i])+ str(ds2_i["target_span"][i])
            ds2_set.add(v)
        for i in range(len(ds_i["entities"])):
            v = str(ds_i["entities"][i]) + str(ds_i["entity_tags"][i]) + str(ds_i["entity_spans"][i])+ str(ds_i["target_span"][i])
            ds_set.add(v)
    # 交集 == len(ds_set) == len(ds2_set)
        inter_set = ds2_set.intersection(ds_set)
        assert len(inter_set) == len(ds_set) == len(ds2_set)
    return num_ent

In [ ]:
from fastNLP import seq_len_to_mask
from fastNLP.core.utils import _move_dict_value_to_device
from fastNLP import SequentialSampler
from fastNLP import DataSetIter
import torch.nn.functional as F
import time

@torch.no_grad()
def get_update_loss(tgt_tokens, tgt_seq_len, pred):
    """
    :param tgt_tokens: bsz x max_len, 包含了的[sos, token, eos]
    :param pred: bsz x max_len-1 x vocab_size
    :return:
    """
    tgt_seq_len = tgt_seq_len - 1
    time1 = time.time()
    mask = seq_len_to_mask(tgt_seq_len, max_len=tgt_tokens.size(1) - 1).eq(0)
    time_res = time.time() - time1
    tgt_tokens = tgt_tokens[:, 1:].masked_fill(mask, -100)
    loss = F.cross_entropy(target=tgt_tokens, input=pred.transpose(1,2), reduction='none').sum(-1) / tgt_seq_len
    return loss,time_res

@torch.no_grad()
def get_loss_all(ds, model):
    # 下标或者add/delete操作会变化
    time_sum = 0
    sampler = SequentialSampler()
    batch = DataSetIter(batch_size=1024, dataset=ds, sampler=sampler)
    model.eval()
    loss_all = torch.empty(0, device='cuda')
    for id,i in enumerate(batch):
        i[0]["update_tree"]=True
        _move_dict_value_to_device(i[0],i[-1],device=torch.device('cuda'))
        tgt_tokens = i[0]['tgt_tokens']
        tgt_seq_len = i[-1]['tgt_seq_len']
        
        pred = model(**i[0])['pred']
        
        batch_loss,time_res = get_update_loss(tgt_tokens, tgt_seq_len, pred)
        time_sum += time_res
        loss_all = torch.cat((loss_all, batch_loss))
    print(time_sum)
    return loss_all

@torch.no_grad()
def updata_func(current_tree1, is_ordered_key1, ds_,loss_all, pipe):
    # ds_是分化后的训练数据集
    current_tree = deepcopy(current_tree1)
    assert current_tree[list(current_tree.keys())[-1]]["ds_indexes"][-1] == len(ds_) - 1,f'{current_tree[list(current_tree.keys())[-1]]["ds_indexes"][-1]} {len(ds_) - 1}'
    is_ordered_key = deepcopy(is_ordered_key1)
    ds2 = DataSet()
    split_list = ["entities","entity_tags","entity_spans","tgt_tokens","target_span","tgt_seq_len"]
    add_list = ["entities","entity_tags","entity_spans","target_span"] # 直接将最后一个加在hard尾巴的key
    for key in current_tree.keys():
        if len(current_tree[key]["ds_indexes"]) == 1 or current_tree[key]['position'] == current_tree[key]['stop_position']:
            assert key in is_ordered_key,f'{key},{current_tree[key],{ds_[0]}}'
        if key in is_ordered_key:
            # 已经排序完毕的key
            assert len(current_tree[key]['ds_indexes']) == 1 and current_tree[key]['position'] == current_tree[key]['stop_position']

            ds2.append(ds_[current_tree[key]['ds_indexes'][0]])
            current_tree[key]["ds_indexes"] = [len(ds2)-1] # 更新一下数据索引
            continue

        position = current_tree[key]['position']
        ds_indexes = current_tree[key]['ds_indexes']
        loss_unordered = loss_all[ds_indexes]
        hard_node_id = ds_indexes[loss_unordered.argmax()]
        ds_hard = ds_[hard_node_id]
        
        hard_value = {}
        for i in split_list:
            hard_value[i] = ds_hard[i]
        hard_value["tgt_tokens"] = hard_value["tgt_tokens"][:-1]

        new_ds_indexes=[]

        for id in ds_indexes:
            if id==hard_node_id:
                continue

            add_value = {}
            for i in add_list:
                add_value[i] = ds_[id][i][-1:]

            add_value["tgt_tokens"] = ds_[id]["tgt_tokens"][position:]

            add_ds = deepcopy(ds_hard)
            for i in add_list:
                add_ds[i] = hard_value[i] + add_value[i]
            add_ds["tgt_tokens"] = hard_value["tgt_tokens"] + add_value["tgt_tokens"]
            add_ds["tgt_seq_len"] = len(add_ds["tgt_tokens"])

            ds2.append(add_ds)

            new_ds_indexes.append(len(ds2)-1)
        
        current_tree[key]["ds_indexes"] = new_ds_indexes
        if len(new_ds_indexes) == 1:
            new_position = len(ds2[-1]["tgt_tokens"]) # 理论上=stop_position
            assert new_position == current_tree[key]["stop_position"], f'{new_position} {current_tree[key]["stop_position"]} {ds2[-1]["tgt_tokens"]}, {len(ds2)}'
        else:
            new_position = len(hard_value["tgt_tokens"])

        current_tree[key]["position"] = new_position
        if new_position == current_tree[key]["stop_position"]:
            assert len(new_ds_indexes)==1,f'分化数据归一才能算排序完成！！！'
            is_ordered_key.add(key)
        ds2.set_ignore_type('target_span', 'entities')
        ds2.set_pad_val('tgt_tokens', 1)  # 设置为eos所在的id
        ds2.set_pad_val('src_tokens', pipe.tokenizer.pad_token_id)

        ds2.apply_field(lambda x: len(x), field_name='src_tokens', new_field_name='src_seq_len')
        ds2.apply_field(lambda x: len(x), field_name='tgt_tokens', new_field_name='tgt_seq_len')
        ds2.set_input('tgt_tokens', 'src_tokens', 'src_seq_len', 'tgt_seq_len', 'first')
        ds2.set_target('tgt_tokens', 'tgt_seq_len', 'target_span', 'entities')
    return current_tree, is_ordered_key, ds2

In [ ]:
def update_tree(ds, model, current_tree, is_ordered_key, pipe):
    # 通过get_loss_all获得在训练集上的每一条loss
    loss_all = get_loss_all(ds, model)
    # 再通过updata_func获得新的ds,current_tree, is_ordered_key
    new_current_tree, new_is_ordered_key, new_ds = updata_func(current_tree, is_ordered_key, ds,loss_all, pipe)
    return new_current_tree, new_is_ordered_key, new_ds

In [ ]:
print(len(ds_))

In [ ]:
while len(is_ordered_key) < len(current_tree):
    x = len(is_ordered_key)
    current_tree, is_ordered_key, ds_ = update_tree(ds_, model2, current_tree, is_ordered_key, pipe)
    print(f"{x} -> {len(is_ordered_key)} max: {len(current_tree)}")

In [ ]:
from fastNLP import DataSet,DataSetIter
from fastNLP import SequentialSampler
ds_ = DataSet()
ds_ =ds.load("/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/caches/ds_final_11_18_2.pt")
sampler = SequentialSampler()
batch = DataSetIter(batch_size=1, dataset=ds_, sampler=sampler)
# model2.eval()

In [ ]:
split_list = ["entities","entity_tags","entity_spans","tgt_tokens","target_span","tgt_seq_len"]
name_list = ["raw_words", "src_tokens","first","src_seq_len"]
# num=0
# for id,i in enumerate(ds):
#     num+=1
#     if len(i["entity_tags"]) > -1:
#         for j in split_list:
#             try:
#                 i[j].sort()
#                 ds_[id][j].sort()
#                 assert(i[j]==ds_[id][j])
#             except:
#                 assert type(i[j])==type(1) and i[j]==ds_[id][j]
#         print("\n")
# print(num)
for id,i in enumerate(ds):
    for j in name_list:
        assert i[j] == ds_[id][j]

In [ ]:
print(len(ds))
for id,i in enumerate(batch):
    src_tokens=i[0]["src_tokens"]
    src_seq_len=i[0]["src_seq_len"]
    first=i[0]["first"]
    if len(ds[id]["entity_tags"]) > 20:
        # split_list = ["entities","entity_tags","entity_spans","tgt_tokens","target_span","tgt_seq_len"]
        # for j in split_list:
            # print(j," ",i[j])
        x=i[0]["tgt_tokens"].to('cuda')
        # print(ds[id]["entities"])
        # print(ds[id]["entity_tags"])
        # print(len(ds[id]["entities"])==len(ds[id]["entity_tags"]))
        y = model2.predict(src_tokens.to('cuda'), src_seq_len.to('cuda'), first.to('cuda'))['pred']
        
        if x.equal(y) != False:
            print(ds[id]["tgt_tokens"])
            print("tgt_tokens: ",x)
            print("pred: ",y)
        print("\n")    

In [ ]:
label_ids = list(mapping2id.values())
metric = Seq2SeqSpanMetric(1, num_labels=len(label_ids), target_type=target_type)
model2.eval()

for i in batch:
    src_tokens=i[0]["src_tokens"]
    src_seq_len=i[0]["src_seq_len"]
    first=i[0]["first"]
    pred = model2.predict(src_tokens.to('cuda'), src_seq_len.to('cuda'), first.to('cuda'))['pred']
    res = metric.evaluate(i[1]["target_span"], pred.cpu(), i[1]["tgt_tokens"])
print(metric.get_metric())
